In [146]:
from google.colab import drive
drive._mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install pandas==1.0.5

In [147]:
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import bz2
import json
from urllib.parse import urlparse
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import glob
import pickle

### Reading everything 


In [ ]:
def read_yearly(year):
    PATH = '/content/drive/MyDrive/ADA_/Fox_NY_'+ str(year) +'/'
    nb = 1
    df2 = []
    dirs = glob.glob(os.path.join(PATH, "*.pkl"))
    
    for files in dirs:
        df1 = pd.read_pickle(PATH + str(nb)+ '_' + str(year) + '_' + 'FoxNYtimes.pkl')
        df2.append(df1)
        nb += 1
    return df2

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
df_foxNy = pd.DataFrame()

for y in years:
  df_foxNy = df_foxNy.append(read_yearly(y))

In [ ]:
#Removes duplicated rows
df_foxNy = df_foxNy.drop_duplicates(subset=['quoteID', 'quotation', 'date', 'urls'], keep="first")

In [ ]:
df_Fox = df_foxNy[df_foxNy['urls'].str.contains("foxnews")]
df_NY = df_foxNy[df_foxNy['urls'].str.contains("nytimes")]

In [ ]:
#Length of Fox News quotations
len(df_Fox)

708383

In [ ]:
#Length of NYT quotations
len(df_NY)

894838

## Making pickles of quotations by year for each journal

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
Common_nouns_years = pd.DataFrame()
for y in years:
  df_foxNy = pd.DataFrame()

  df_foxNy = df_foxNy.append(read_yearly(y))
  df_foxNy = df_foxNy.drop_duplicates(subset=['quoteID', 'quotation', 'date', 'urls'], keep="first")

  df_Fox = df_foxNy.quotation[df_foxNy['urls'].str.contains("foxnews")]
  df_NY = df_foxNy.quotation[df_foxNy['urls'].str.contains("nytimes")]

  df_Fox.to_pickle('/content/drive/Shareddrives/ADA/Quotations_Fox_NY/' + str(y) + '_' + 'Fox_quotations.pkl')
  df_NY.to_pickle('/content/drive/Shareddrives/ADA/Quotations_Fox_NY/'+ str(y) + '_' + 'NY_quotations.pkl')


## Number of quotations per journal

In [ ]:
#Length words in each journal
years = [2015, 2016, 2017, 2018, 2019, 2020]
PATH = '/content/drive/Shareddrives/ADA/Quotations_Fox_NY/'
vectorizer = CountVectorizer()

for y in years:
  Common_nouns = pd.DataFrame(columns = [['Fox_words','Fox_count','NY_words','NY_count']])
  df_Fox = pd.read_pickle(PATH + str(y) + '_' + 'Fox_quotations.pkl')
  df_NY = pd.read_pickle(PATH + str(y) + '_' + 'NY_quotations.pkl')

  Quotations_Fox_count += len(df_Fox)
  Quotations_NY_count += len(df_NY)

  print(Quotations_Fox_count)
  print(Quotations_NY_count)

### Most commom words 

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')

#Collections
from collections import Counter

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models.phrases import Phrases
from gensim.corpora import Dictionary

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def processing (doc):

  STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS
  processed_docs = list()

  #ent = [token.ent_type_ for token in doc if token.ent_type_ != '']

  # Keep only words (no numbers, no punctuation).
  # Lemmatize tokens, remove punctuation and remove stopwords.
  doc = [str(token.lemma_) for token in doc if token.is_alpha and not token.is_stop ]

  # Remove common words from a stopword list and keep only words of length 3 or more.
  doc = [token for token in doc if token not in STOPWORDS and len(token) > 2]

      #processed_docs.extend(doc)
      #entities.extend(ent)
  return(doc)

In [ ]:
def preprocess_pipe(texts):
    preproc_pipe = list()
    is_noun = lambda pos: pos[:2] == 'NN'

    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.extend(processing(doc))

    nouns = [word for (word, pos) in nltk.pos_tag(preproc_pipe) if is_noun(pos)]
    df_nouns = Counter(nouns).most_common(30)
    return (df_nouns,count)

In [ ]:
#Get common words over all years

Common_nouns_years = pd.DataFrame()

years = [2015, 2016, 2017, 2018, 2019, 2020]
PATH = '/content/drive/Shareddrives/ADA/Quotations_Fox_NY/'

for y in years:
  Common_nouns = pd.DataFrame(columns = [['Fox_words','Fox_count','NY_words','NY_count']])
  df_Fox = pd.read_pickle(PATH + str(y) + '_' + 'Fox_quotations.pkl')
  df_NY = pd.read_pickle(PATH + str(y) + '_' + 'NY_quotations.pkl')

  Common_nouns[['Fox_words','Fox_count']] = preprocess_pipe(df_Fox)[0]
  Common_nouns[['NY_words','NY_count']] = preprocess_pipe(df_NY)[0]

  Common_nouns_years = Common_nouns_years.append(Common_nouns)

In [ ]:
Common_nouns_years.to_pickle('/content/drive/Shareddrives/ADA/Common_nouns_all.pkl')
common = pd.read_pickle('/content/drive/Shareddrives/ADA/Common_nouns_all.pkl')